# 추천시스템
- 하나의 콘텐츠를 선택했을 때, 선택된 컨텐츠와 연관된 추천 컨텐츠를 제공하는 서비스 시스템
- 대표적으로 사용하는 데이터
    1. 사용자가 선택한 컨텐츠
    2. 사용자가 실제 구매와 연관된 액션을 했는가 여부
    3. 사용자가 선택한 컨텐츠에 내린 평가 데이터
    4. 사용자가 스스로 작성한 자신의 취향
    5. 사용자가 무엇을 클릭했는지...

## 1. 추천시스템 유형
- 협업 필터링 vs 콘텐츠 기반 필터링
    1. 협업 필터링: 최근접이웃 협업 vs 잠재요인 협업
        + 협업필터링이란, 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터 기반으로 사용자가 아직 평가하지 않은 아이템을 '예측 평가'하는 것
        + 최근접 이웃 협업, 잠재요인 협업 모두 사용자-평점 매트릭스만으로 사용자에게 추천함
        + index는 user, columns는 item1, item2 등으로 구성되어야함. pandas.pivot_table() 이용해 변경해야함
        + 예시. 영화 {범죄도시1: 평점 8.5점, 범죄도시2: 평점 9점, 범죄도시3: 평점 8점} => 범죄도시 4: 평점 8.75로 예측
        + 최근접이웃 협업: 사용자 기반 vs 아이템 기반
            * 사용자 기반: 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
            * 아이템 기반: 이 상품을 선택한 다른 고개들은 다음 상품도 구매했습니다.
        + 잠재요인 협업: 사용자-아이템 평점 매트리스 속에 숨어 있는 잠재 요인을 추출해 추천하는 기법
    2. 콘텐츠 기반 필터링: 사용자가 특정 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 장르 아이템 추천


### 잠재요인 행렬 분해 코드
- SGD를 이용해 행렬분해를 수행
- 분해하려는 행렬 R
- P와 Q로 분해 후 다시 P와 Q.T의 내적으로 예측행렬을 만듬

In [5]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K=3

# P와 Q 행렬의 크기를 지정하고 정규분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [6]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 Null이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_index = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_index = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_index, y_non_zero_index]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_index, y_non_zero_index]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [7]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j , R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
lr = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구하기
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularlization을 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + lr*(eij*Q[j, :] - r_lambda*P[i, :])
        Q[j, :] = Q[j, :] + lr*(eij*P[i, :] - r_lambda*Q[j, :])
        rmse = get_rmse(R, P, Q, non_zeros)
        if step % 50 == 0:
            print(f"[행렬분해 중] iteration step: {step}, rmse: {rmse}")

[행렬분해 중] iteration step: 0, rmse: 3.261355059488935
[행렬분해 중] iteration step: 0, rmse: 3.26040057174686
[행렬분해 중] iteration step: 0, rmse: 3.253984404542389
[행렬분해 중] iteration step: 0, rmse: 3.2521583839863624
[행렬분해 중] iteration step: 0, rmse: 3.252335303789125
[행렬분해 중] iteration step: 0, rmse: 3.251072196430487
[행렬분해 중] iteration step: 0, rmse: 3.2492449982564864
[행렬분해 중] iteration step: 0, rmse: 3.247416477570409
[행렬분해 중] iteration step: 0, rmse: 3.241926055455223
[행렬분해 중] iteration step: 0, rmse: 3.2400454107613084
[행렬분해 중] iteration step: 0, rmse: 3.240166740749792
[행렬분해 중] iteration step: 0, rmse: 3.2388050277987723
[행렬분해 중] iteration step: 50, rmse: 0.5003190892212749
[행렬분해 중] iteration step: 50, rmse: 0.5001616291326989
[행렬분해 중] iteration step: 50, rmse: 0.4989960120257809
[행렬분해 중] iteration step: 50, rmse: 0.498848345014583
[행렬분해 중] iteration step: 50, rmse: 0.4989518925663175
[행렬분해 중] iteration step: 50, rmse: 0.4983323683009099
[행렬분해 중] iteration step: 50, rmse: 0.4984148489378

In [8]:
# 분해된 P와 Q 함수를 P*Q.T로 예측행렬 만들기
pred_matrix = np.dot(P,Q.T)
print('예측행렬:\n', np.round(pred_matrix, 3))

예측행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


### TEST - 콘텐츠 기반 필터링 실습
- data from TMDB 5000
- https://www.kaggle.com/tmdb/tmdb-movie-metadata

In [53]:
# library
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv("./data/tmdb_5000_movies.csv")
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [54]:
# 콘텐츠 기반 필터링 추천에 사용할 칼럼만 추출해 새로운 데이터프레임 생성
cols = ['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']
movie_df = movies[cols]
movie_df

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6,238,14.269792,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...
4800,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0,6,1.444476,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],When ambitious New York attorney Sam is sent t...


In [55]:
movie_df['genres'][0]['name']

TypeError: string indices must be integers

In [56]:
# list 내 dict를 반복해 읽을 수 있게 만들어주는 함수
from ast import literal_eval
movie_df['genres'] = movie_df['genres'].apply(literal_eval)
movie_df['keywords'] = movie_df['keywords'].apply(literal_eval)
movie_df[['genres', 'keywords']]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
...,...,...
4798,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 5616, 'name': 'united states–mexico ba..."
4799,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",[]
4800,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 248, 'name': 'date'}, {'id': 699, 'nam..."
4801,[],[]


In [57]:
# genres, keyword가 keyword: value 형태의 data로 구성되어있음. genres는 genres만, keyword는 keyworkd만 입력하도록 변경
movie_df['genres'] = movie_df['genres'].apply(lambda x : [y['name'] for  y in x])
movie_df['keywords'] = movie_df['keywords'].apply(lambda x : [y['name'] for  y in x])
movie_df[['genres', 'keywords']]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi..."
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i..."
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel..."
...,...,...
4798,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap..."
4799,"[Comedy, Romance]",[]
4800,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi..."
4801,[],[]


In [58]:
movie_df

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[Action, Crime, Thriller]",6.6,238,14.269792,"[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...
4799,72766,Newlyweds,"[Comedy, Romance]",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",7.0,6,1.444476,"[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],When ambitious New York attorney Sam is sent t...


- CountVectorizer는 276개의 개별 단어 피처로 구성된 피처 벡터 행렬을 만들어주는 것


In [59]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 적용 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movie_df['genres_literal'] = movie_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movie_df['genres_literal'])

movie_df['genres_literal']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

- cosine_similarity()는 코사인 유사도 행렬을 만들어줌
- 상관관계 행렬과 유사한 형태를 보임

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])


(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


- 유사도가 높은 인덱스 순서로 sort!

In [61]:
genre_sim_sorted_index = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_index)

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 ...
 [4800 3809 1895 ... 2229 2230    0]
 [4802 1594 1596 ... 3204 3205    0]
 [4802 4710 4521 ... 3140 3141    0]]


### 장르 콘텐츠 필터링을 이용한 영화 추천
- 함수명: find_sim_movie()
- 인자: movie_df, 레코드별 장르 코사인 유사도 인덱스 가지고 있는 genre_sim_sorted_index, 영화 제목, 추천할 영화 건수
- 결과: 추천영화 정보를 가진 DataFrame 반환

In [62]:
def find_sim_movie(df, sorted_index, title_name, top_n=10):
    # 인자로 입력된 movie_df에서 title 칼럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]

    # title_named을 가진 DataFrame의 Index 객체를 ndaaray로 반환하고
    # sorted_index 인자로 입력된 genre_sim_sorted_index 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_index[title_index, :(top_n)]

    # 추출된 top_n index 출력. top_n index는 2차원 데이터임
    # dataframe 에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [63]:
similar_movies = find_sim_movie(movie_df, genre_sim_sorted_index, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1
